In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
data = pd.read_csv('House_Rent_Dataset.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop(['Posted On', 'Floor', 'Area Locality',"Point of Contact"], axis=1)
data.head()